## Diffusive Flux Matrix Implementation



The pressure is define like this:
\begin{equation}
    \label{pressure}
    \textit{p} (\mathbf{U}) = (\gamma-1) \bigg(\textit{e}_{tot}-\frac{|\mathbf{m}|^2}{2 \rho} \bigg)
\end{equation}

The shear stress tensor is defined as follow.


\begin{equation}
    \tau_{ij}(\mathbf{U}) = \frac{\mu}{\rho}\bigg(\frac{\partial m_i}{\partial x_j}+\frac{\partial m_j}{\partial x_i}\bigg)-\frac{2\mu}{3\rho}\bigg(\frac{\partial m_k}{\partial x_k}\bigg)\delta_{ij}-\frac{\mu}{\rho^2}\bigg(m_i \frac{\partial\rho}{\partial x_j}+m_j \frac{\partial \rho}{\partial x_i}\bigg)+\frac{2\mu}{3\rho^2}\bigg(m_k \frac{\partial \rho}{\partial x_k}\bigg)\delta_{ij} \quad i,j,k<{d}
\end{equation}

Substituting $\mathbf{H}(\mathbf{U})= \frac{\partial \mathbf{U}}{\partial x_j}$, which is a $\textit{(dim+2)*(dim)}$ matrix, and $\mathbf{U}$ in the equation:

\begin{equation}
    \tau_{ij}(\mathbf{U}) = \frac{\mu}{U_0}\bigg(H_{i+1,j}+H_{j+1,i}\bigg)-\frac{2\mu}{3U_0}\bigg(H_{k+1,k}\bigg)\delta_{ij}-\frac{\mu}{U_0^2}\bigg(U_{i+1} H_{0,j}+U_{j+1}H_{0,i}\bigg)+\frac{2\mu}{3U_0^2}\bigg(U_{k+1} H_{0,k}\bigg)\delta_{ij} \quad i,j,k<{d}
\end{equation}

The heat flux $q_i(\mathbf{U})$ is:

\begin{equation}
    q_i(\mathbf{U}) = \frac{\lambda \textit{e}_{tot}}{\rho^2 \textit{c}_v}\frac{\partial \rho}{\partial x_i}-\frac{\lambda m_jm_j}{\rho^3 \textit{c}_v}\frac{\partial \rho}{\partial x_i}+\frac{\lambda m_j}{\rho^2 \textit{c}_v}\frac{\partial m_j}{\partial x_i}-\frac{\lambda}{\rho \textit{c}_v}\frac{\partial \textit{e}_{tot}}{\partial x_i}
\end{equation}

Which substituting $\mathbf{H}$ and $\mathbf{U}$ becomes:
\begin{equation}
    q_i(\mathbf{U}) = \frac{\lambda U_{d+1}}{U_0^2 \textit{c}_v}H_{0,i}-\frac{\lambda U_{j+1}U_{j+1}}{U_0^3 \textit{c}_v}H_{0,i}+\frac{\lambda U_{j+1}}{U_0^2 \textit{c}_v}H_{j+1,i}-\frac{\lambda}{U_0 \textit{c}_v}H_{d+1}\quad i,j<d
\end{equation} 

NB: $\rho$ is the first term of the vector of unknwons $\mathbf{U}$ and $\textit{e}_{tot}$ is the last one (Ug\[dim+1\]).
Remember to consider them as Unknowns and not fluid parameters.

Defining $\mathbf{G}$ as:

\begin{equation}
    \label{Gmat}
    \mathbf{G}_j(\mathbf{U}) = \big(0,-\tau_{ji}, -\frac{m_i}{\rho}\tau_{ij}+q_j\big)^T \quad \quad i,j<{d}
\end{equation}

$\mathbf{K}_{kj}(\mathbf{U})$ ($\textit{(d+2)*(d+2)*d*d}$ matrix) and can be derived from this expression:

\begin{equation}
    \label{K}
    \frac{\partial\mathbf{G}_j(\mathbf{U})}{\partial x_j} = -\frac{\partial}{\partial x_k}\bigg(\mathbf{K}_{kj}(\mathbf{U})\frac{\partial \mathbf{U}}{\partial x_j}\bigg)
\end{equation}

As: 

\begin{equation}
    \mathbf{K}_{ij} = -\frac{\partial \mathbf{G}_j(\mathbf{U})}{\partial \mathbf{H}_k}
\end{equation}

Each (dim+2)*(dim+2) matrix a $\mathbf{K}_{ij}$ is:

\begin{equation}
    \mathbf{K}_{i,j} = -\frac{\partial G_{l,j}}{\partial H_{m,k}} \quad \quad l,m <{dim+2}
\end{equation}


In [ ]:
'''
from KratosMultiphysics import *

from sympy import *
from sympy_fe_utilities import *
import pprint

## Computation of the Diffusive Matrix
def computeK(dofs,params):
    print("\nCompute Diffusive Matrix\n")
    dim = params["dim"]				# spatial dimensions
    ## Unknown fields definition
    H = DefineMatrix('H',dim+2,dim)		# Gradient of U
    G = DefineMatrix('G',dim+2,dim)		# Diffusive Flux matrix 
    tau = DefineMatrix('tau',dim,dim)		# Shear stress tensor for Newtonian fluid
    q = DefineVector('q',dim)			# Heat flux vector
    
    ## Other simbols definition
    Cv = params["c_v"]				# Specific Heat at Constant volume
    Cp = params["c_p"]				# Specific Heat at Constant Pressure
    y = params["gamma"]				# Gamma (Cp/Cv) 
    nu = params["nu"]		         	# Kinematic viscosity (mu/rho)
    mu  = params["mu"]         			# Dynamic viscosity 
    l = params["lambda"]			# Thermal Conductivity of the fluid
        
    ## Data interpolation to the Gauss points
    Ug = dofs

    ## Pgauss - Pressure definition
    pg = (y-1)*Ug[dim+1]
    for i in range(0,dim):
        pg += (y-1)*(-Ug[i+1]*Ug[i+1]/(2*Ug[0]))
    
    ## Tau - Shear stress tensor definition

    for i in range(0,dim):
        for j in range(i,dim):#NOT SURE
            if i!=j:
               tau[i,j] = mu/Ug[0]*(H[i+1,j]+H[j+1,i])-mu/Ug[0]**2*(Ug[i+1]*H[0,j]+Ug[j+1]*H[0,i])
            if i==j:
               tau[i,j]= 2*mu/Ug[0]*H[i+1,i]-2*mu/Ug[0]**2*Ug[i+1]*H[0,i]
               for k in range(0,dim):
                   tau[i,j]+= -2*mu/(3*Ug[0])*H[k+1,k]+2*mu/(3*Ug[0]**2)*Ug[k+1]*H[0,k]
    
    for i in range(1,dim):
        for j in range(0,dim-1):
            if j!=i:
               tau[i,j] = tau[j,i]
    '''         
    for i in range(0,dim):
        for j in range(0,dim):
            print(i,j,"=	",tau[i,j])
    '''              

     ## q - Heat flux vector definition
    for i in range(0,dim):
        q[i] = l*Ug[dim+1]/(Ug[0]**2*Cv)*H[0,i]-(l*H[dim+1,i])/(Ug[0]*Cv)
        for j in range(0,dim):
            q[i] += -l*Ug[j+1]**2/(Cv*Ug[0]**3)*H[0,i]+l/(Ug[0]**2*Cv)*Ug[j+1]*H[j+1,i] 
    #NB!!!There is an error in the definition of q[i] in the research proposal. The second term of the equation has an opposite sign!!!NB#
       
    ## G - Diffusive Matrix definition 
    for j in range(0,dim):
        G[0,j]= 0 			#Mass equation related
       
    for i in range(1,dim+1):
        for j in range(0,dim):
            G[i,j]=-tau[i-1,j]		#Moment equation related
    
    for j in range(0,dim):
        G[dim+1,j] = q[j]
        for k in range(0,dim):
            G[dim+1,j] += -Ug[k+1]*tau[k,j]/Ug[0]
    
    ## K - Jacobian Diffusive Matrix definition
    K = []				#Final 5*5*3 tensor 			
    # k:index of H(moving over colomns), j:index of G(moving over colomns)	
      
    for k in range(0,dim):
        ksmall = []			#Intermediate 5*5*3 tensor
        for j in range(0,dim):
            tmp = DefineMatrix('tmp',dim+2,dim+2)
            for l in range(0,dim+2):
                for m in range(0,dim+2):
                    tmp[l,m] = diff(-G[l,j],H[m,k])
                    #print("\n\n",l,m,"=",tmp[l,m],"\n\n")
            
            ksmall.append(tmp)
        
        K.append(ksmall)     
    return K
    
## Printing the Diffusive Matrix
def printK(K,params):
    dim = params["dim"]
    ksmall = []
    tmp = []
    print("The diffusive matrix is:\n")
    for k in range (0,dim):
        ksmall = K[k]
        for j in range(0,dim):
      	    tmp = ksmall[j]
      	    #print(tmp)
      	    for i in range(0,dim+2):
                for l in range(0,dim+2):  
                    print("K[",k,",",j,",",i,",",l,"]=",tmp[i,l],"\n")
       
    return 0
'''